In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
sys.path.insert(0, 'code/')

In [4]:
import h5py
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchsummaryX import summary

from mac import MACNetwork
from utils import load_vocab
from datasets import ClevrDataset, collate_fn

In [5]:
from config import cfg_from_file, __C, cfg

cfg_from_file('cfg/local.yml')
__C.CUDA = False
__C.GPU_ID = '-1'
vocab = load_vocab(cfg)
# cfg.TRAIN.RECV_OBJECTS = False

code/config.py:83: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))


In [6]:
model = MACNetwork(cfg=cfg, max_step=4, vocab=vocab)
model.load_state_dict(torch.load('/Users/sebamenabar/Documents/vanilla_mac.pth', map_location='cpu')['model'])
# model(b['image'], b['question'], b['question_length'])

<All keys matched successfully>

In [7]:
dataset = h5py.File('/Users/sebamenabar/Documents/datasets/CLEVR/data/train_features.h5', 'r')['features']

In [8]:
total_size = 200
train_size = 100
model.eval()
with torch.no_grad():
    X = model.input_unit.stem(torch.from_numpy(dataset[:total_size]))

In [9]:
e = np.eye(196)

X_train = X[:train_size].numpy().reshape(train_size * 196, 512)
y_train = np.tile(e, train_size).T

test_size = total_size - train_size
X_test = X[train_size:].numpy().reshape(test_size * 196, 512)
y_test = np.tile(e, test_size).T

print('train', X_train.shape, y_train.shape)
print('test', X_test.shape, y_test.shape)

# y_train = np.where(y_train)[1]
# y_test = np.where(y_test)[1]

train (19600, 512) (19600, 196)
test (19600, 512) (19600, 196)


In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [54]:
kmodel = Sequential()
# kmodel.add(Dense(256, input_dim=512, activation='relu'))
# kmodel.add(Dense(256, input_dim=256, activation='relu'))
kmodel.add(Dense(196, input_dim=512, activation='softmax',
                 use_bias=False,
                  kernel_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0001),
                  bias_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0001),
                 ))
kmodel.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(lr=0.001, decay=1e-6),
    metrics=['accuracy'],
)

In [55]:
kmodel.fit(X_train, y_train, epochs=10, batch_size=100, shuffle=True, validation_data=(X_test, y_test))

Train on 19600 samples, validate on 19600 samples
Epoch 1/10
19600/19600 [==============================] - 2s 119us/step - loss: 1.5667 - acc: 0.8235 - val_loss: 0.5049 - val_acc: 0.9945
Epoch 2/10
19600/19600 [==============================] - 2s 78us/step - loss: 0.4589 - acc: 0.9993 - val_loss: 0.4398 - val_acc: 0.9986
Epoch 3/10
19600/19600 [==============================] - 2s 81us/step - loss: 0.4004 - acc: 0.9999 - val_loss: 0.3792 - val_acc: 0.9993
Epoch 4/10
19600/19600 [==============================] - 2s 81us/step - loss: 0.3406 - acc: 1.0000 - val_loss: 0.3176 - val_acc: 0.9994
Epoch 5/10
19600/19600 [==============================] - 2s 86us/step - loss: 0.2812 - acc: 1.0000 - val_loss: 0.2599 - val_acc: 0.9998
Epoch 6/10
19600/19600 [==============================] - 2s 79us/step - loss: 0.2277 - acc: 1.0000 - val_loss: 0.2104 - val_acc: 0.9998
Epoch 7/10
19600/19600 [==============================] - 2s 82us/step - loss: 0.1832 - acc: 1.0000 - val_loss: 0.1704 - val_ac

In [56]:
weights1 = kmodel.get_weights()
weights1

[array([[ 2.2358629e-03, -1.4154413e-02, -9.0380563e-05, ...,
         -8.4808422e-03,  7.2930530e-03, -2.9074756e-04],
        [-4.2515923e-04, -1.4523097e-03,  2.0638179e-05, ...,
         -6.3934707e-04, -1.8544253e-03, -8.4042287e-05],
        [-1.2840978e-05, -2.6644695e-05,  4.0164043e-05, ...,
         -2.3540099e-04, -2.0521661e-02,  5.0598662e-02],
        ...,
        [ 4.0479682e-02, -1.1271374e-02,  9.7249049e-06, ...,
          1.6816927e-03, -3.1261958e-02, -5.1750692e-05],
        [-4.0984727e-03, -3.6190096e-03, -2.2749231e-05, ...,
         -9.9391065e-05,  3.3799175e-04, -1.2946275e-02],
        [ 4.4808030e-04, -1.2940196e-04,  1.9662242e-05, ...,
          2.8878288e-02,  1.5310259e-03, -3.6968591e-03]], dtype=float32)]

In [15]:
e = np.eye(196)

train_size = 100
X2_train = dataset[:train_size].reshape(train_size * 196, 1024)
y2_train = np.tile(e, train_size).T

test_size = 100
X2_test = dataset[train_size:train_size + test_size].reshape(test_size * 196, 1024)
y2_test = np.tile(e, test_size).T

print('train', X2_train.shape, y2_train.shape)
print('test', X2_test.shape, y2_test.shape)

train (19600, 1024) (19600, 196)
test (19600, 1024) (19600, 196)


In [16]:
k2model = Sequential()
# k2model.add(Dense(256, input_dim=1024, activation='relu'))
# kmodel.add(Dense(256, input_dim=256, activation='relu'))
k2model.add(Dense(196, input_dim=1024, activation='softmax',
                  kernel_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0001),
                  bias_regularizer=keras.regularizers.l1_l2(l1=0.0001, l2=0.0001),
                 ))
k2model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
k2model.fit(X2_train, y2_train, epochs=2, batch_size=100, shuffle=True, validation_data=(X2_test, y2_test))

Train on 19600 samples, validate on 19600 samples
Epoch 1/2
19600/19600 [==============================] - 3s 141us/step - loss: 3.6136 - acc: 0.8278 - val_loss: 2.5989 - val_acc: 0.9984
Epoch 2/2
19600/19600 [==============================] - 2s 125us/step - loss: 2.2738 - acc: 0.9996 - val_loss: 2.0868 - val_acc: 0.9997


In [18]:
weights = k2model.get_weights()

In [19]:
weights

[array([[-1.2631699e-01,  7.0139565e-02, -5.3971546e-04, ...,
          3.2462232e-02,  3.0657114e-03,  5.8356319e-02],
        [-1.7055456e-01, -1.5830781e-03, -1.9613721e-03, ...,
          4.0400486e-02, -1.1706071e-03,  6.7951560e-02],
        [-1.1000342e-01, -2.1936090e-03,  4.8164674e-04, ...,
          2.7934385e-02,  1.5969898e-03,  4.2893395e-02],
        ...,
        [-3.5136319e-03,  1.0517805e-04, -1.1601908e-03, ...,
          7.1837708e-02,  7.2785141e-04, -1.5070821e-01],
        [-3.6273822e-02, -2.4802044e-02, -1.1450350e-02, ...,
          1.6930515e-02, -9.8239724e-03, -1.5758118e-01],
        [ 1.6387844e-02, -5.7152208e-02, -8.5496224e-02, ...,
          1.0043573e-02,  6.2840365e-02, -1.1441434e-01]], dtype=float32),
 array([-1.16822369e-01,  1.58842281e-02,  6.49235919e-02,  1.05016485e-01,
        -1.41443357e-01, -1.27707124e-01, -2.51767598e-02, -3.57584730e-02,
         1.10784024e-01, -6.19348176e-02, -9.79764313e-02, -1.34518534e-01,
         6.23929612e-0